In [1]:
from Diagnosis import df
import pandas as pd


Loading processed data from pickle file...


In [2]:
df.head()

,Date,Product,Channel,EMR#,Consultation #,Mobile #,Member #,Type,Status,Patient Gender,...,Doctor,Chief Complaint,Diagnosis,Advice,REFERRED,Lab Tests,Medicine,Clinic Name,Prescrption json,ICD Code
0,2020-05-15,M-Swasth Doctor - FREE,CashPor,1,1,7571029662,21:01:84:01:25,CALL,DONE,M,...,SATISH,Other,.,referred to higher centre,NO,NaN,NO,NaN,"{""pres"":[]}",.
1,2021-06-10,M-Swasth Doctor - FREE,CashPor,1,47576,7571029662,21:01:84:01:25,CALL,DONE,M,...,KADAMBARI,Other,Dysmenorrhea and menorrhagia,Advised not miss a single tablet of Ovral l\nD...,NO,NaN,YES,NaN,"{""pres"":[{""mName"":""Tab Ovral L"",""m"":"""",""a"":"""",...",NONE
2,2021-06-11,M-Swasth Doctor - FREE,CashPor,1,48043,7571029662,21:01:84:01:25,CALL,DONE,M,...,NIKHIL,Other,C/o Weakness with loss of appetite,"Eat healthy food , green vegetables and fruits...",NO,NaN,YES,NaN,"{""pres"":[{""mName"":""Multiprex Cap"",""m"":""1"",""a"":...",NONE
3,2021-06-11,M-Swasth Doctor - FREE,CashPor,1,48603,7571029662,21:01:84:01:25,CALL,DONE,M,...,ravindra,Other,pain,Take nutritious diet\nDo Excercise regularly\n...,NO,CBC,YES,NaN,"{""pres"":[{""mName"":""Paracetamol 500mg Tab"",""m"":...",R50
4,2021-06-16,M-Swasth Doctor - FREE,CashPor,1,52282,7571029662,21:01:84:01:25,CALL,DONE,M,...,Harinder Pal,Other Pain,Gall Stones,Referred to nearby hospital for surgical consu...,YES,Ultrasound,NO,NaN,"{""pres"":[]}",K87


In [3]:
import os
import pandas as pd

# Define the path to the pickle file
pickle_file_path = 'clinic_cleaned.pkl'

# Check if the pickle file already exists
if os.path.exists(pickle_file_path):
    df = pd.read_pickle(pickle_file_path)
    print(f"Pickle file loaded from: {pickle_file_path}")
else:


    column_to_clean = 'Diagnosis'

    df[column_to_clean] = df[column_to_clean].str.replace('?', '', regex=False)
    df[column_to_clean] = df[column_to_clean].str.replace('C/O', '', regex=False)
    df[column_to_clean] = df[column_to_clean].str.replace('C/o', '', regex=False)
    df[column_to_clean] = df[column_to_clean].str.replace('c/o', '', regex=False)
    df[column_to_clean] = df[column_to_clean].str.replace('H/O', '', regex=False)
    df[column_to_clean] = df[column_to_clean].str.replace('H/O', '', regex=False)
    df[column_to_clean] = df[column_to_clean].str.replace('K/C/O HTN', '', regex=False)
    df[column_to_clean] = df[column_to_clean].replace(r'\\n', ',', regex=True).replace(r'\n', ',', regex=True)
    df[column_to_clean] = df[column_to_clean].str.replace('u/E', '', regex=False)
    df[column_to_clean] = df[column_to_clean].str.replace('U/E', '', regex=False)

    # Deleting column
    column_to_delete = 'Diagonis final'
    if column_to_delete in df.columns:
        df.drop([column_to_delete], axis=1, inplace=True)
    else:
        print(f"Column '{column_to_delete}' not found in the DataFrame.")

    df[column_to_clean] = df[column_to_clean].str.lower()

    # Save the cleaned DataFrame to a pickle file
    df.to_pickle(pickle_file_path)
    print(f"Pickle file saved to: {pickle_file_path}")

# Display the updated DataFrame


Pickle file loaded from: clinic_cleaned.pkl


In [4]:
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df = df.dropna(subset=['Date'])
df = df[df['Date'].dt.year >= 2024]

In [5]:
df['Diagnosis'] = df.apply(
    lambda row: row['Chief Complaint'] if pd.isnull(row['Diagnosis']) and isinstance(row['Chief Complaint'], str) and 
    row['Chief Complaint'].lower() != 'other' and row['Chief Complaint'].lower() != 'other pain' else row['Diagnosis'], 
    axis=1
)


In [6]:
df["Diagnosis"].dropna(inplace=True)


In [7]:
df = df.sort_values(by='Date', ascending=True)


In [8]:
df.columns

Index(['Date', 'Product', 'Channel', 'EMR#', 'Consultation #', 'Mobile #',
       'Member #', 'Type', 'Status', 'Patient Gender', 'FEMALE DOC ONLY',
       'Doctor', 'Chief Complaint', 'Diagnosis', 'Advice', 'REFERRED',
       'Lab Tests', 'Medicine', 'Clinic Name', 'Prescrption json', 'ICD Code'],
      dtype='object')

In [9]:
df['Diagnosis'].value_counts()

Diagnosis
urti                                                                          35828
weakness                                                                      31393
cough                                                                         20405
weakness                                                                      20372
knee pain                                                                     17860
                                                                              ...  
heel pain , shoulder pain                                                         1
injury to shoulder since 10yrs                                                    1
anoraxia /weakness                                                                1
low backache with tingling sensation legs - since 1 month,htn                     1
upper respiratory tract infection c weakness  c diabetes c hypothyroidism         1
Name: count, Length: 318349, dtype: int64

In [10]:
df['Diagnosis'] = df['Diagnosis'].str.strip().str.lower()
df= df[df['Diagnosis'] != '']

In [11]:
df['Diagnosis'].value_counts()

Diagnosis
weakness                                                                     52753
urti                                                                         43466
cough                                                                        37441
knee pain                                                                    25604
headache                                                                     21912
                                                                             ...  
fever , cough, cold since 1 week                                                 1
typhoid,gastritis,weakness                                                       1
numbness rt thigh                                                                1
fever ,body pain ,back pain                                                      1
upper respiratory tract infection c weakness  c diabetes c hypothyroidism        1
Name: count, Length: 297904, dtype: int64

In [12]:
import pandas as pd
import re

# Create a regex pattern to split by multiple delimiters
delimiters = [',', '/', '&', ' and ', ' , ', 'with']
pattern = '|'.join(map(re.escape, delimiters))

# Function to split and explode the Diagnosis column


def split_and_explode(df, column, pattern):
    # Split the column by the pattern and create a new DataFrame with exploded values
    s = df[column].str.split(pattern, expand=True).stack()
    s = s.str.strip()  # Remove leading/trailing whitespaces
    # Adjust index to match the original DataFrame
    s.index = s.index.droplevel(-1)
    return df.drop(columns=[column]).join(s.rename(column))


# Apply the function to the DataFrame
result_data = split_and_explode(df, 'Diagnosis', pattern)

# Remove duplicates and reset the index
result_data = result_data.drop_duplicates().reset_index(drop=True)



In [13]:
result_data['Diagnosis'].value_counts()

Diagnosis
weakness                                                                     246682
cough                                                                        152530
gastritis                                                                     89980
headache                                                                      78070
fever                                                                         75091
                                                                              ...  
itchy lesion over thigh                                                           1
cold - allergic in nature                                                         1
eyeache since 2 days                                                              1
rhinorrhea - since 2 days                                                         1
upper respiratory tract infection c weakness  c diabetes c hypothyroidism         1
Name: count, Length: 112643, dtype: int64

In [14]:
# reverse_mapping = {
#     'weakness': ['weakness', 'generalized weakness', 'generalized body weakness', 'generalised weakness', 'generalised body pain', 'generalised body ache', 'weaknes', 'weekness', 'weakness.', 'bodyache weakness', 'bodypain weakness', 'headache weakness', 'fever weakness', 'weakness since 8days',"bodyache,weakness","1 weakness"],
#     'cough': ['cough', 'dry cough', 'cough.', 'coughing', 'cough cold', 'cough x3d', 'productive cough', 'non productive', 'fever cough', 'fever cough cold', 'cough cold since 2days', 'wet cough', 'acute cough', 'cough.', 'flu like symptoms-cough', 'fever cold cough'],
#     'gastritis': ['gastritis', 'acute gastritis', 'gas', 'hyperacidity', 'gerd', 'indigestion', 'acidity', 'acidity.', 'acid reflux'],
#     'headache': ['headache', 'headache.', 'head ache', 'migraine'],
#     'fever': ['fever', 'viral fever', 'seasonal fever', 'fever x3d', 'mild fever.', 'pyrexia'],
#     'urti': ['urti', 'upper respiratory tract infection', 'urti features', 'uri', 'upper respiratory tract infection c weakness', 'upper respiratory tract infection c weakness c anorexia'],
#     'knee pain': ['knee pain', 'knee pain b', 'kneepain', 'knee pain [ weakness ]', 'knee pain', 'knee joint pain', 'knee-pain', 'knee arthralgia'],
#     'cold': ['cold', 'common cold', 'common cold weakness', 'cold cough', ' cold', 'cold and cough'],
#     'body pain': ['body pain', 'body pain.', 'body pain', 'body pain and. weakness', 'generalised body pain', 'generalized body pain', 'body pains', 'body pain weakness', 'generalized body ache'],
#     'back pain': ['back pain', 'low back pain', 'low backache', 'low back ache', 'lba', 'backache', 'lower back pain', 'lower back ache', 'backpain', 'pain back', 'pain in back', 'pain lower back', 'lumbago'],
#     'leg pain': ['leg pain', 'pain in legs', 'l leg pain', 'left leg pain', 'legpain', 'right leg pain', 'pain in lower limb', 'leg.', 'leg', 'pain in leg'],
#     'myalgia': ['myalgia', 'myalgia [ weakness ]', 'myalgia b'],
#     'ringworm': ['ringworm', 'ring worm', 'ringworm infection', 'ring worm infection', 'tinea', 'tinea corporis', 'tinea cruris'],
#     'anorexia': ['anorexia', 'loss of appetite', 'anorexia'],
#     'leucorrhoea': ['leucorrhoea', 'leucorrhea', 'leukorrhea'],
#     'hypertension': ['hypertension', 'htn', 'high bp'],
#     'hypotension': ['hypotension', 'low bp'],
#     'toothache': ['toothache', 'tooth ache', 'tooth pain', 'dental caries'],
#     'chest pain': ['chest pain'],
#     'itching': ['itching', 'pruritus', 'pruritis', 'itching over body', 'itching all over body', 'itchiness', 'skin itching'],
#     'malaise': ['malaise', 'generalized malaise'],
#     'shoulder pain': ['shoulder pain', 'pain shoulder', 'right shoulder pain', 'left shoulder pain'],
#     'joint pain': ['joint pain', 'arthralgia', 'arthalgia', 'joint pain', 'arthralgia knee', 'polyarthralgia'],
#     'fungal infection': ['fungal infection'],
#     'arthritis': ['arthritis', 'osteoarthritis'],
#     'rhinitis': ['allergic rhinitis', 'rhinitis'],
#     'gastrointestinal symptoms': ['gastrointestinal symptoms', 'gastroenteritis'],
#     'diabetes': ['diabetes', 'diabetes mellitus', 'dm', 't2dm'],
#     'skin problems': ['skin problems', 'skin problem', 'skin infection', 'skin allergy', 'eczema', 'dermatitis', 'contact dermatitis', 'dandruff', 'xeroderma'],
#     'anemia': ['anaemia', 'anemia', 'anemia'],
#     'dizziness': ['dizziness', 'giddiness'],
#     'vomiting': ['vomiting', 'vomitting','vomit','pukish'],
#     'nausea': ['nausea'],
#     'swelling': ['swelling', 'oedema', 'edema'],
#     'dehydration': ['dehydration'],
#     'hair loss': ['hairfall', 'hair fall'],
#     'urinary tract infection': ['uti', 'urinary tract infection'],
#     'body pain': ['body pain', 'generalised body pain', 'generalized body pain'],
#     'generalized pain': ['generalized pain'],
#     'visual acuity': ['visual acuity', 'refractive error'],
#     'pain': ['pain', 'other pain', 'body pain', 'bodypain', 'pain body', 'pain abdomen', 'pain in abdomen', 'abdominal pain', 'pain in body', 'generalised body pain', 'generalised pain'],
#     'sore throat': ['sore throat', 'pharyngitis'],
#     'ear pain': ['ear pain', 'earache'],
#     'neck pain': ['neck pain'],
#     'allergy': ['allergy', 'skin allergy', 'itching over body', 'itching all over body'],
#     'nutritional deficiency': ['nutritional deficiency', 'vitamin deficiency'],
#     'hypothyroidism': ['hypothyroidism'],
#     'diabetes': ['diabetes', 'diabetes mellitus'],
#     'spondylosis': ['spondylosis', 'cervical spondylosis']
#     # Add more mappings as needed
# }
print(3)


3


In [15]:
# def map_diagnosis(diagnosis):
#     """Map diagnosis to its canonical form using the reverse_mapping dictionary."""
#     if isinstance(diagnosis, str):
#         diagnoses = [d.strip().lower() for d in diagnosis.split(',')]
#     elif isinstance(diagnosis, list):
#         diagnoses = [d.strip().lower() for d in diagnosis]
#     else:
#         return diagnosis

#     mapped_diagnoses = []
#     for d in diagnoses:
#         mapped = False
#         for canonical_form, synonyms in reverse_mapping.items():
#             if d in [s.lower() for s in synonyms]:
#                 mapped_diagnoses.append(canonical_form)
#                 mapped = True
#                 break
#         if not mapped:
#             mapped_diagnoses.append(d)
    
#     return ', '.join(mapped_diagnoses)
print(6)


6


In [16]:
import pandas as pd
import numpy as np
from rapidfuzz import process, fuzz

def create_canonical_forms(df, column_name, threshold=80):
    """
    Create a dictionary of canonical forms based on the most common diagnoses using RapidFuzz.
    
    Args:
    df (pandas.DataFrame): The input DataFrame
    column_name (str): The name of the column containing diagnoses
    threshold (int): The similarity threshold for fuzzy matching
    
    Returns:
    dict: A dictionary mapping similar diagnoses to their canonical forms
    """
    # Split multi-diagnoses and create a flat list, handling NaN values
    all_diagnoses = [
        diagnosis.strip().lower() 
        for diagnoses in df[column_name].dropna().astype(str).str.split(',') 
        for diagnosis in diagnoses
    ]
    
    # Get unique diagnoses sorted by frequency
    unique_diagnoses = pd.Series(all_diagnoses).value_counts().index.tolist()
    
    canonical_forms = {}
    mapped_diagnoses = set()
    
    for diagnosis in unique_diagnoses:
        if diagnosis not in mapped_diagnoses:
            try:
                similar_diagnoses = process.extract(diagnosis, unique_diagnoses, limit=None, scorer=fuzz.ratio)
                for match in similar_diagnoses:
                    if len(match) >= 2:
                        similar, score = match[0], match[1]
                        if score >= threshold and similar not in mapped_diagnoses:
                            canonical_forms[similar] = diagnosis
                            mapped_diagnoses.add(similar)
                    else:
                        print(f"Unexpected match format for diagnosis '{diagnosis}': {match}")
            except Exception as e:
                print(f"Error processing diagnosis '{diagnosis}': {str(e)}")
    
    return canonical_forms

def map_diagnosis(diagnosis, canonical_forms):
    """Map a single diagnosis to its canonical form."""
    if pd.isna(diagnosis):
        return np.nan
    return canonical_forms.get(str(diagnosis).strip().lower(), diagnosis)

def process_diagnoses(df, column_name):
    """Process the diagnoses column and map to canonical forms."""
    # Create canonical forms
    canonical_forms = create_canonical_forms(df, column_name)
    
    # Function to process a single cell (which may contain multiple diagnoses)
    def process_cell(cell):
        if pd.isna(cell):
            return np.nan
        diagnoses = [d.strip() for d in str(cell).split(',')]
        mapped_diagnoses = [map_diagnosis(d, canonical_forms) for d in diagnoses]
        return ', '.join(filter(lambda x: not pd.isna(x), mapped_diagnoses))
    
    # Apply the processing to the entire column
    df[column_name] = df[column_name].apply(process_cell)
    
    return df

# Assuming you have a DataFrame named 'df' with a 'Diagnosis' column
# df = pd.read_csv('your_data.csv')  # Uncomment this line if you need to load the data



In [17]:
# Assuming you have a DataFrame named 'df' with a 'Diagnosis' column
# df = pd.read_csv('your_data.csv')  # Uncomment this line if you need to load the data

# Process the diagnoses
result_data = process_diagnoses(result_data, 'Diagnosis')

# Display the updated value counts
print(result_data['Diagnosis'].value_counts())



Diagnosis
weakness                                                        256730
cough                                                           154070
gastritis                                                        91866
headache                                                         82422
fever                                                            76260
                                                                 ...  
pain in lower abdomen  periodical c dysmenorrhoea c weakness         1
pv spotting but no pregnancy.                                        1
feet  c anxiety  c anorexia                                          1
pain in neck decreased vision                                        1
had a h                                                              1
Name: count, Length: 43390, dtype: int64

Diagnosis Mapping:


In [37]:
def remove_invalid_diagnoses(df, column_name='Diagnosis'):
    """
    Remove blank entries and single-letter entries from the specified column.
    
    Args:
    df (pandas.DataFrame): The input DataFrame
    column_name (str): The name of the column to process (default is 'Diagnosis')
    
    Returns:
    pandas.DataFrame: The DataFrame with invalid entries removed
    """
    # Replace empty strings, whitespace-only strings, and single-letter entries with NaN
    df[column_name] = df[column_name].replace(r'^\s*$', np.nan, regex=True)
    df[column_name] = df[column_name].replace(r'^\s*.\s*$', np.nan, regex=True)
    
    # Remove rows where the specified column is NaN
    df = df.dropna(subset=[column_name])
    
    # Strip leading and trailing whitespace from the remaining entries
    df[column_name] = df[column_name].str.strip()
    
    return df


In [38]:
result_data = remove_invalid_diagnoses(result_data)


C:\Users\siidh\AppData\Local\Temp\ipykernel_16884\789178421.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = df[column_name].replace(r'^\s*$', np.nan, regex=True)
C:\Users\siidh\AppData\Local\Temp\ipykernel_16884\789178421.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = df[column_name].replace(r'^\s*.\s*$', np.nan, regex=True)
C:\Users\siidh\AppData\Local\Temp\ipykernel_16884\789178421.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

In [41]:
result_data["Diagnosis"].value_counts()

Diagnosis
weakness                                  256730
cough                                     154070
gastritis                                  91866
headache                                   82422
fever                                      76260
                                           ...  
cough  swalling in hand                        1
lumbar spondylosis  not on treatment           1
albinism                                       1
lump in the axila 1mth lymphadenopathy         1
had a h                                        1
Name: count, Length: 43349, dtype: int64

In [42]:
result_data.to_pickle("Diagnosis_filtered.pkl")

In [18]:
# result_data['Diagnosis'] = result_data['Diagnosis'].apply(map_diagnosis)


In [19]:
# result_data['Diagnosis'].value_counts().to_csv("1.csv")


ENDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDD